In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

2023-09-24 14:21:23.598234: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-24 14:21:23.603108: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 14:21:23.708879: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 14:21:23.710176: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 14:21:25.457239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<IPython.core.display.Javascript object>

# Functions and definitions

### Data preprocessing

In [3]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [5]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [6]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

## Sequence helper

In [7]:
def generate_sequences(dataset, timesteps):
    dataset["x_train"], dataset["y_train"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_train"], columns=x.columns),
                dataset["y_train"],
            ],
            axis=1,
        ).values,
        timesteps,
    )

    dataset["x_test"], dataset["y_test"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_test"], columns=x.columns),
                dataset["y_test"],
            ],
            axis=1,
        ).values,
        timesteps,
    )
    return dataset

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [8]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    estimator_params=None,
    imputer_params=None,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [9]:
class LSTM(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=16, activation="relu"))
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [10]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [11]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [12]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Dataframe structure to save the results

In [13]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_ii",
    "Features": "Chemical + Mineralogical + CS1 + CS3",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "LSTM",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [14]:
df = pd.read_csv("../../../../../data/processed/partner_ii/Type-I-II.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [15]:
df_copy = df.drop(
    [
        # Properties
        "loi",
        "water_demand",
        "setting_initial",
        "blaine",
        "sieve_32um",
        "sieve_45um",
        "CS7",
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [16]:
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [17]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - LSTM

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (410, 37)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 41 samples each
    2. 90% train (369 samples each fold)
    3. 10% test (41 samples each fold)
<b>Total:</b> 100 models<br>

In [18]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"], axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate(
        LSTM,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        estimator_params=None,
        imputer_params={"strategy": "median"},
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



2023-09-24 14:21:29.028136: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-24 14:21:29.028257: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2023-09-24 14:21:29.028270: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2023-09-24 14:21:29.028532: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.125.6
2023-09-24 14:21:29.028580: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.125.6
2023-09-24 14:21:29.028598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.125.6


TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.668 (0.059)
MAE: 1.351 (0.055)
MAPE: 0.032 (0.001)
R2: 0.494 (0.034)


******
[TEST]
******
RMSE: 1.936 (0.146)
MAE: 1.529 (0.064)
MAPE: 0.036 (0.002)
R2: 0.308 (0.126)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.651 (0.217)
MAE: 1.336 (0.190)
MAPE: 0.032 (0.004)
R2: 0.499 (0.138)


******
[TEST]
******
RMSE: 2.189 (0.256)
MAE: 1.735 (0.191)
MAPE: 0.041 (0.005)
R2: 0.134 (0.200)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.387 (0.092)
MAE: 1.117 (0.082)
MAPE: 0.027 (0.002)
R2: 0.654 (0.046)


******
[TEST]
******
RMSE: 2.019 (0.188)
MAE: 1.600 (0.134)
MAPE: 0.038 (0.003)
R2: 0.268 (0.183)


Minutes Elapsed:  52.13492402235667


<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (410, 37)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 82 samples
    2. 50% train (41 samples each fold)
    3. 50% test (41 samples each fold)
<b>Total:</b> 5 models<br>

In [19]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 6.555 (0.964)
MAE: 5.129 (0.849)
MAPE: 0.122 (0.020)
R2: -9.597 (2.100)


******
[TEST]
******
RMSE: 10.516 (3.654)
MAE: 8.735 (3.377)
MAPE: 0.211 (0.091)
R2: -29.466 (15.537)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.909 (0.371)
MAE: 1.549 (0.330)
MAPE: 0.037 (0.008)
R2: 0.020 (0.442)


******
[TEST]
******
RMSE: 12.935 (5.473)
MAE: 10.860 (5.052)
MAPE: 0.263 (0.134)
R2: -49.939 (35.909)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.808 (0.372)
MAE: 1.459 (0.301)
MAPE: 0.035 (0.007)
R2: 0.089 (0.520)


******
[TEST]
******
RMSE: 22.383 (18.362)
MAE: 20.200 (18.151)
MAPE: 0.494 (0.468)
R2: -311.253 (490.978)


Minutes Elapsed:  13.925262900193532


<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (410, 37)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 40, 77, 114, 152, 189, 226, 263, 301, 338 samples each fold
    2. Test: 37 samples each fold
<b>Total:</b> 10 models<br>

In [20]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.940 (1.376)
MAE: 2.273 (1.005)
MAPE: 0.054 (0.024)
R2: -2.039 (3.087)


******
[TEST]
******
RMSE: 5.881 (3.403)
MAE: 4.796 (2.915)
MAPE: 0.114 (0.067)
R2: -10.758 (12.458)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.963 (0.417)
MAE: 1.605 (0.383)
MAPE: 0.038 (0.009)
R2: -0.054 (0.455)


******
[TEST]
******
RMSE: 5.721 (3.310)
MAE: 4.624 (2.590)
MAPE: 0.110 (0.061)
R2: -10.306 (15.878)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.798 (0.446)
MAE: 1.474 (0.401)
MAPE: 0.035 (0.009)
R2: 0.089 (0.543)


******
[TEST]
******
RMSE: 6.278 (5.275)
MAE: 5.184 (4.473)
MAPE: 0.123 (0.104)
R2: -16.270 (26.654)


Minutes Elapsed:  41.219182888666786


<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (410, 37)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 328
    2. Test: 82
<b>Total:</b> 1 model<br>

In [21]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        # Data Splitting
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)

        # Sequence Splitting
        data_train = pd.concat(
            [
                pd.DataFrame(x_train, columns=x.columns),
                y_train.reset_index(drop=True),
            ],
            axis=1,
        ).values
        data_test = pd.concat(
            [
                pd.DataFrame(x_test, columns=x.columns),
                y_test.reset_index(drop=True),
            ],
            axis=1,
        ).values

        x_train, y_train = split_sequences(data_train, timesteps)
        x_test, y_test = split_sequences(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", LSTM())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.658 (0.060)
MAE: 1.346 (0.053)
MAPE: 0.031 (0.001)
R2: 0.349 (0.048)


******
[TEST]
******
RMSE: 2.445 (0.094)
MAE: 1.900 (0.088)
MAPE: 0.049 (0.002)
R2: -0.216 (0.093)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.449 (0.101)
MAE: 1.168 (0.092)
MAPE: 0.027 (0.002)
R2: 0.502 (0.069)


******
[TEST]
******
RMSE: 2.830 (0.437)
MAE: 2.298 (0.340)
MAPE: 0.059 (0.009)
R2: -0.789 (0.511)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.416 (0.059)
MAE: 1.140 (0.023)
MAPE: 0.027 (0.001)
R2: 0.529 (0.039)


******
[TEST]
******
RMSE: 3.165 (0.464)
MAE: 2.610 (0.417)
MAPE: 0.068 (0.011)
R2: -1.297 (0.686)


Minutes Elapsed:  11.154018060366313


<IPython.core.display.Javascript object>

In [22]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features Model            Cross Validation  \
                                                                             
0   Chemical + Mineralogical + CS1 + CS3  LSTM  Blocking Time Series Split   
1   Chemical + Mineralogical + CS1 + CS3  LSTM  Blocking Time Series Split   
2   Chemical + Mineralogical + CS1 + CS3  LSTM  Blocking Time Series Split   
3   Chemical + Mineralogical + CS1 + CS3  LSTM           Out of time Split   
4   Chemical + Mineralogical + CS1 + CS3  LSTM           Out of time Split   
5   Chemical + Mineralogical + CS1 + CS3  LSTM           Out of time Split   
6   Chemical + Mineralogical + CS1 + CS3  LSTM              Repeated KFold   
7   Chemical + Mineralogical + CS1 + CS3  LSTM              Repeated KFold   
8   Chemical + Mineralogical + CS1 + CS3  LSTM              Repeated KFold   
9   Chemical + Mineralogical + CS1 + CS3  LSTM           Time Series Split   
10  Chemical + Mineralogical + CS1 + CS3  LSTM           Time Series Split   
11  Chemical + Mineralogical + CS1 + CS3  LSTM           Time Series Split   

   Timesteps  RMSE Test              MAE Test            MAPE Test            \
                   mean        std       mean        std      mean       std   
0          1  10.515999   3.653658   8.734744   3.376575  0.211426  0.090603   
1          7  12.935464   5.472602  10.859606   5.051639  0.263212  0.133569   
2         14  22.382665  18.361761  20.199922  18.151006  0.494425  0.468388   
3          1   2.444654   0.094075   1.900335   0.088258  0.049169  0.002327   
4          7   2.829834   0.437049   2.297792   0.339971  0.059495  0.009111   
5         14   3.164513   0.463824   2.610357   0.417034  0.068053  0.010852   
6          1   1.935621   0.146094   1.529128   0.063846  0.036412  0.001516   
7          7   2.189219   0.256447   1.735323   0.191324  0.041314  0.004504   
8         14   2.019013   0.187788   1.600196   0.133653  0.038316  0.003046   
9          1   5.880901   3.402949   4.795943   2.915267  0.113688  0.067463   
10         7   5.720803   3.309965   4.624433   2.590305  0.110117  0.060623   
11        14   6.278110   5.275217   5.184120   4.472980  0.123001  0.103714   

       R2 Test              
          mean         std  
0   -29.465773   15.536658  
1   -49.938665   35.908541  
2  -311.253336  490.977736  
3    -0.216237    0.092902  
4    -0.789052    0.511389  
5    -1.296511    0.685585  
6     0.307560    0.125736  
7     0.133878    0.200032  
8     0.267639    0.183011  
9   -10.758167   12.458176  
10  -10.305544   15.877877  
11  -16.270011   26.654310

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [23]:
path = "../../../../../reports/results/local_models/partner_ii/type-I-II/full/"
filename = "lstm_results_full_6.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [24]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_ii/type-I-II/grouped/"
filename = "lstm_results_grouped_6.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>